# **Import The nessary library**

In [2]:
import pandas as pd
import pickle

# **Read The Data set**

In [3]:
df = pd.read_csv('data/questions.csv', sep=',')

In [4]:
df.head()

,post_id,scheme,category,title,answer
0,0,Stand Up India buissness,business loan for backward,Nature,The objective of the Stand-Up India scheme is ...
1,1,Stand Up India buissness,business loan for backward,Purpose,The objective of the Stand-Up India scheme is ...
2,2,Stand Up India buissness,business loan for backward,Objective,The objective of the Stand-Up India scheme is ...
3,3,Stand Up India buissness,business loan for backward,Introduction of the scheme,The objective of the Stand-Up India scheme is ...
4,4,Stand Up India buissness,business loan for backward,Quantum of loan,The loans provided under this scheme will rang...


# **Information about data**

In [5]:
df.columns

Index(['post_id', 'scheme', 'category', 'title', 'answer'], dtype='object')

In [6]:
# check a   null value
df.isnull().sum()


post_id     0
scheme      0
category    0
title       0
answer      2
dtype: int64

In [7]:
# fill a  null value
df.ffill(inplace=True)

In [8]:
# after filling checking null value check
df.isnull().sum()

post_id     0
scheme      0
category    0
title       0
answer      0
dtype: int64

In [9]:
#data types of columns
df.dtypes

post_id      int64
scheme      object
category    object
title       object
answer      object
dtype: object

In [10]:
df['scheme'].value_counts()

scheme
Pradhan Mantri Suraksh Bima Yojna                        23
Pradhan Mantri Awas Yojana                               20
Atal Pension Yojana                                      19
Pradhan Mantri Fasal Bima Yojana                         19
vida lakshmi yojana                                      19
general Insurance                                        18
Rashtriya Swasthya Bima Yojana                           17
Aam admi bima yojana                                     16
Scheme For Liberation and Rehabilitation of Scavenger    16
Pradhan Mantri Jeevan Jyoti BIMA Yojana                  16
Pradhan Mantri Vaya Vandana Yojana                       15
Varishtha Pension bima yojana                            15
Prime Ministers Rozgar Yojana                            14
Pradhan Mantri Mudra Yojana                              13
Employment State Insurance Scheme                        13
National Social Assistance Programme insurance           13
Stand Up India buissness         

In [11]:
df['category'].value_counts()

category
health insurance                      41
Health insurance                      23
Home loan                             20
pension scheme                        19
agriculture crop insurance            19
education loan                        19
general insurance faq                 18
accident disablity death insurance    16
house loan                            16
life insurance                        16
pension scheme insurance              15
pension insurance                     15
buissness loan                        14
small business loan                   13
insurance                             13
business loan for backward            13
accidental and death insurance        12
senior citizen insurance              11
genaral home loan faq                 10
Name: count, dtype: int64

In [12]:
df['title'].value_counts()

title
Introduction of the scheme                             12
Eligibility Criteria                                   11
Objective                                              10
Nature                                                  9
Purpose                                                 9
                                                       ..
What is the nature of Pradhan Mantri Awas Yojana ?      1
What is the purpose of Pradhan Mantri Awas Yojana ?     1
What are the programs this scheme addresses ?           1
Who is eligible for this scheme ?                       1
Expenses Considered for Loan                            1
Name: count, Length: 239, dtype: int64

# **Train Module**

In [13]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Load your CSV dataset
data = df
user_messages = data["title"]
chatbot_responses = data["answer"]

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Tokenize and encode your dataset
input_ids = []
attention_masks = []

for i in range(len(user_messages)):
    encoded_dict = tokenizer.encode_plus(
        user_messages[i],
        chatbot_responses[i],
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors="pt",
    )
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])

# Convert lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor([1] * len(data))  # Assuming positive examples

# Split data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42
)

# Create data loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()  # Assuming binary classification

# Training loop
model.train()
for epoch in range(3):
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained("your_chatbot_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\0630\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad 

In [14]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained("your_chatbot_model")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [15]:
input_text = ["bhima yojana"]

# Tokenize and encode the input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=128, pad_to_max_length=True, return_tensors="pt")
attention_mask = input_ids.ne(0)  # Create attention mask (1 for tokens, 0 for padding)

# Make predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# If you're performing binary classification, you can convert the logits to probabilities:
probabilities = torch.sigmoid(logits)

# Depending on your specific use case, you can now interpret the model's output for your chatbot application.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [16]:
probabilities

tensor([[0.2200, 0.8394]])

In [17]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained("your_chatbot_model")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [18]:
from transformers import BertForSequenceClassification, BertTokenizer
import json

# Load the configuration from the JSON file
with open('your_chatbot_model/config.json', 'r') as file:
    config = json.load(file)

# Create a tokenizer
tokenizer = BertTokenizer.from_pretrained(config["_name_or_path"])

# # # Create a BERT model using the configuration
# model = BertForSequenceClassification.from_pretrained(config["_name_or_path"], config=config)

# Now you can use the `tokenizer` to tokenize text and the `model` for various NLP tasks, such as chatbot responses or text classification.

# Example of using the tokenizer and model
text = "Personal Loan"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)

In [19]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.4742,  1.8961]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
logits = outputs.logits  # Get the classification scores
predicted_label_id = logits.argmax().item()  # Get the predicted label ID
class_names = df['answer']  # Replace with your class names
predicted_label = class_names[predicted_label_id]
print(f"Predicted Label: {predicted_label}")

Predicted Label: The objective of the Stand-Up India scheme is to facilitate bank loans between 10 lakh and 1 Crore to at least one Scheduled Caste (SC) or Scheduled Tribe (ST) borrower and at least one woman borrower per bank branch for setting up a greenfield enterprise. This enterprise may be in manufacturing, services or the trading sector. In case of non-individual enterprises at least 51% of the shareholding and controlling stake should be held by either an SC/ST or Woman entrepreneur.


In [21]:
df.columns

Index(['post_id', 'scheme', 'category', 'title', 'answer'], dtype='object')

In [22]:
result = data[df['title'].str.contains(text, case=False)]

In [23]:
data = df
inp = 'What would be the premium payable'
fil_data = data[data['title'].str.contains(inp, case=False)]
if fil_data.empty:
    print("Not Responce")
else:
    text = fil_data.sort_values(by='title')[:1]
    Ans = text['answer'].tolist()

In [24]:
Ans

['Premium payable is Rs.12/- per annum per member.']

In [25]:
df['title'][:100]

0                                                Nature
1                                               Purpose
2                                             Objective
3                            Introduction of the scheme
4                                       Quantum of loan
                            ...                        
95    Can those families whose names are not on the ...
96        What would be the benefits under the scheme ?
97                  What would be the premium payable ?
98                       How will the premium be paid ?
99            Who will offer or administer the scheme ?
Name: title, Length: 100, dtype: object

In [26]:
import re

In [27]:
inp = 'purpose'

In [28]:
# Define a function to generate related text
def generate_related_text(input_text, dataset):
    related_texts = []
    for text in dataset:
        if re.search(input_text, text, re.IGNORECASE):
            related_texts.append(text)
    return related_texts

# Generate related text for the input
related_text = generate_related_text(inp, df['title'])

# Print the related text
for text in related_text:
    print(text)

Purpose
Purpose
Purpose
Purpose
What is the purpose of MUDRA?
What is the purpose of scheme ?
What is the nature or purpose of the scheme ?
What is the purpose of Pradhan Mantri Awas Yojana ?
What is the purpose of the scheme
Purpose
Purpose
Purpose
Purpose
what is the purpose of the scheme
What is the purpose of the scheme
What is the purpose of the scheme
Purpose
What is the purpose of the scheme


In [29]:
df.columns

Index(['post_id', 'scheme', 'category', 'title', 'answer'], dtype='object')

In [34]:
user_input = input("You: ")
if user_input == 'hi' or 'hello':
    result = 'hello can you please tell your name'
    print("Chatbot: Response logits:", result)
    user_input = input("You: ")
    print("hello", user_input)
    print('wich bank information yoy want')
    print('1) RBI')
    print('2) NABARD')
    user_input = input("You: ")
if user_input == 'RBI' or 'rbi':
    print("enter wich scheme information you want")
    print(df['scheme'].value_counts())
    # Load the configuration from the JSON file
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        with open('your_chatbot_model/config.json', 'r') as file:
            config = json.load(file)
        result = data[df['title'].str.contains(user_input, case=False)][:1]
        print("Chatbot: Response logits:", result)

# You can post-process the response logits to generate human-readable responses.

Chatbot: Response logits: hello can you please tell your name
hello rr
wich bank information yoy want
1) RBI
2) NABARD
enter wich scheme information you want
scheme
Pradhan Mantri Suraksh Bima Yojna                        23
Pradhan Mantri Awas Yojana                               20
Atal Pension Yojana                                      19
Pradhan Mantri Fasal Bima Yojana                         19
vida lakshmi yojana                                      19
general Insurance                                        18
Rashtriya Swasthya Bima Yojana                           17
Aam admi bima yojana                                     16
Scheme For Liberation and Rehabilitation of Scavenger    16
Pradhan Mantri Jeevan Jyoti BIMA Yojana                  16
Pradhan Mantri Vaya Vandana Yojana                       15
Varishtha Pension bima yojana                            15
Prime Ministers Rozgar Yojana                            14
Pradhan Mantri Mudra Yojana                            